In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertModel

In [ ]:
# Load the IMDB dataset
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

In [ ]:
# Instantiate the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Define the input and output layers
input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoding_layer = tokenizer(input_layer, truncation=True, padding=True, return_tensors='tf')
output_layer = bert(encoding_layer)[0][:,0,:]
output_layer = tf.keras.layers.Dropout(0.3)(output_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(output_layer)

In [ ]:
# Define the model and compile it
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_data.batch(32), epochs=2, validation_data=validation_data.batch(32))

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data.batch(32))
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)